In [9]:
from navec import Navec

In [10]:
path = r"S:\Workspace\MORETECH\nlp-workbench\models\navec_news_v1_1B_250K_300d_100q.tar"
navec = Navec.load(path)


In [ ]:
# %pip install torch
# %pip install torchtext


In [ ]:
# import torch
# from slovnet.model.emb import NavecEmbedding

# navec_emb = NavecEmbedding(navec)
# input = torch.tensor([1,2,0])
# output = navec_emb(input)


# output

In [6]:
# %pip install sent2vec

In [ ]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

vectorizer = Vectorizer(pretrained_weights="DeepPavlov/rubert-base-cased")


In [ ]:
sentences = [
    "Рэпер Паша Техник впал в кому после отдыха на вписке с наркотиками",
    "Рэпер Паша Техник впал в кому после вечеринки с наркотиками",
    "Я не люблю кошек",
]

vectorizer.run(sentences)
vectors = vectorizer.vectors[-3:]
dist_1 = spatial.distance.cosine(vectors[0], vectors[1])
dist_2 = spatial.distance.cosine(vectors[0], vectors[2])
print('dist_1: {0}, dist_2: {1}'.format(dist_1, dist_2))
assert dist_1 < dist_2

In [ ]:
#! inference speed is proportional to len(n)^2

In [20]:
# Test inference speed for 100 sentences
import time
start = time.time()
vectorizer.run(sentences * 1000)
end = time.time()
print('Inference time: {0} seconds'.format(end - start))



Inference time: 23.709993600845337 seconds


In [17]:
def get_vectorizer_func(vectorizer):
    def func(sentences):
        vectorizer.run([sentences])
        return vectorizer.vectors[-1]
    return func


In [21]:
# Test inference speed for 100 sentences, but make it parallel
import time
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=4)


# Create a dataframe with 100 sentences
vectorize_func = get_vectorizer_func(vectorizer)
data = sentences * 1000

start = time.time()

# Apply vectorize_func to each sentence in parallel
df = pd.DataFrame(data, columns=['sentence'])
df['vector'] = df['sentence'].parallel_apply(vectorize_func)

end = time.time()
print('Inference time: {0} seconds'.format(end - start))

# Модель для каждого потока грузится в память - memory consumption 2gb * n_workers,
# После этого инференс относительно быстро производится

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


Inference time: 67.2250063419342 seconds


In [ ]:
# За 4 минуты делается инференс на 10000 новостей (при распаралелливании)
# Походу в модели есть свое распараллеливание или она просто крутая, 
# три тысячи записей она делает сама за 23 секунды
# Предположение, все плохо работает из за рейс кондишена в vectorizer... или нет
# В общем без распараллеливания нормально


In [14]:
from turtle import distance
from scipy import spatial
from sent2vec.vectorizer import Vectorizer



class SimilarityModel:
    def __init__(self, weights: str = "DeepPavlov/rubert-base-cased" ) -> None:
        self.vectorizer = Vectorizer(weights)

    def calc_embeddings(self, sentences: list[str]) -> list:
        # Outputs embeddings for each sentence in the list
        # Efficient for batch processing
        vectors = []
        self.vectorizer.run(sentences)
        vectors = self.vectorizer.vectors[-len(sentences):]
        return self.vectors

    def calc_distance_matrix_from_embeddings(self, embeddings1: list, embeddings2: list) -> list:
        distances = []
        for vector in embeddings1:
            distances.append([])
            for vector2 in embeddings2:
                distances[-1].append(spatial.distance.cosine(vector, vector2))
        return distances


    def calc_distance(self, sentence1: str, sentence2: str) -> float:
        # USE FOR TESTS ONLY
        # IT IS UNIMAGINABLY SLOW IN PRODUCTION
        # USE LATER calc_similarity_batch
        self.vectorizer.run([sentence1, sentence2])
        vectors = self.vectorizer.vectors[-2:]
        return spatial.distance.cosine(vectors[0] / len(sentence1), vectors[1] / len(sentence2))

    def calc_distance_batch(self, sentence1: str, sentences: list[str]) -> list:
        self.vectorizer.run([sentence1] + sentences)
        vectors = self.vectorizer.vectors[-(len(sentences) + 1):]
        distances = []
        for vector in vectors[1:]:
            distances.append(spatial.distance.cosine(vectors[0], vector))
        return distances

    def calc_distance_matrix(self, sentences: list[str]) -> list:
        self.vectorizer.run(sentences)
        vectors = self.vectorizer.vectors[-len(sentences):]
        distances = []
        for vector in vectors:
            distances.append([])
            for vector2 in vectors:
                distances[-1].append(spatial.distance.cosine(vector, vector2))
        return distances


distance_matrix = []

if __name__ == "__main__":
    import pandas as pd

    vectorizer = SimilarityModel("sberbank-ai/sbert_large_nlu_ru")
    # vectorizer = SimilarityModel()
    # vectorizer = SimilarityModel("distilbert-base-uncased")

    sentences = [
        "Рэпер Паша Техник впал в кому после отдыха на вписке с наркотиками",
        "Рэпер Паша Техник впал в кому после вечеринки с наркотиками",
        "Я не люблю кошек",
        "Я не люблю собак",
        "Я боюсь собак",
        "Я боюсь Пашу Техника",
        "Путин отмечает 70-летие",
        "Владимир Путин отмечает юбилей: президенту России исполнилось 70 лет",
        "Владимиру Путину — 70 лет! Топ-5 интересных фактов о жизни президента России",
        "Владимиру Путину исполнилось 70 лет",
        "День рождения Путина: президенту России исполнилось 70 лет",
    ]

    from pprint import pprint
    print("="*80)
    pprint(vectorizer.calc_distance(sentences[0], sentences[1]))
    print("="*80)
    pprint(vectorizer.calc_distance_batch(sentences[0], sentences[1:]))
    print("="*80)
    distance_matrix = vectorizer.calc_distance_matrix(sentences)
    # Convert to pandas df
    df = pd.DataFrame(distance_matrix, columns=sentences, index=sentences)
    # print matrix
    for line in distance_matrix:
        print(line)


    pprint(df < 0.1)
    print("="*80)

    # vectors = vectorizer.vectors[-3:]
    # dist_1 = spatial.distance.cosine(vectors[0], vectors[1])
    # dist_2 = spatial.distance.cosine(vectors[0], vectors[2])
    # print('dist_1: {0}, dist_2: {1}'.format(dist_1, dist_2))
    # assert dist_1 < dist_2



Initializing Bert sberbank-ai/sbert_large_nlu_ru
Vectorization done on cpu


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at sberbank-ai/sbert_large_nlu_ru were not used when initializing DistilBertModel: ['encoder.layer.14.attention.output.LayerNorm.weight', 'encoder.layer.19.attention.output.LayerNorm.weight', 'encoder.layer.19.attention.self.query.bias', 'encoder.layer.18.attention.self.value.bias', 'encoder.layer.18.attention.output.dense.bias', 'encoder.layer.17.output.dense.bias', 'encoder.layer.15.attention.self.key.weight', 'encoder.layer.22.attention.self.value.bias', 'encoder.layer.23.output.dense.bias', 'encoder.layer.

0.1140473484992981
[0.06608426570892334,
 0.29877471923828125,
 0.2968018651008606,
 0.31692326068878174,
 0.28476405143737793,
 0.2781725525856018,
 0.17978978157043457,
 0.2909168601036072,
 0.21751892566680908,
 0.2092221975326538]
[0, 0.06608426570892334, 0.29877471923828125, 0.2968018651008606, 0.31692326068878174, 0.28476405143737793, 0.2781725525856018, 0.17978978157043457, 0.2909168601036072, 0.21751892566680908, 0.2092221975326538]
[0.06608426570892334, 0, 0.16427701711654663, 0.16348499059677124, 0.180597186088562, 0.15382540225982666, 0.14378350973129272, 0.12244385480880737, 0.39273422956466675, 0.11091035604476929, 0.12085115909576416]
[0.29877471923828125, 0.16427701711654663, 0, 0.0037273168563842773, 0.012334585189819336, 0.027603864669799805, 0.02644813060760498, 0.15048104524612427, 0.5028277039527893, 0.07137799263000488, 0.12648940086364746]
[0.2968018651008606, 0.16348499059677124, 0.0037273168563842773, 0, 0.008241653442382812, 0.02700483798980713, 0.0289191007614

In [3]:
import matplotlib.pyplot as plt
plt.imshow(distance_matrix, cmap='hot', interpolation='nearest')


: 

: 